In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["about the author", "acknowledgements", "about the book", "table of content"]

In [ ]:
def get_classification(text, eq=None, threshold=0.3):
    assert eq in candidate_labels if eq else True
    if not eq:
        return classifier(text, candidate_labels=candidate_labels)["labels"][0]
    else:
        res = classifier(text, candidate_labels=candidate_labels)
        return res["labels"][0] == eq if res["scores"][0] > threshold else None

In [ ]:
classifier("This is a book about the author", candidate_labels=candidate_labels)

In [ ]:
summary_path = "/Users/andreafavia/development/bookai/Big Feelings.json"

import json

with open(summary_path, "r") as f:
    data = json.load(f)
import pprint

for chapt, text in data.items():
    print("___")
    print(text)